In [1]:
import matplotlib.pyplot as plt
import matplotlib as mp
import matplotlib.animation as animation
import numpy as np
import IPython

from envs.Quadrotor import Quadrotor

In [2]:
m = 1.0
I = 1.0
r = 1.0

quadrotor = Quadrotor(m, I, r)

In [3]:
quadrotor = Quadrotor(m, I, r)
u = np.array([10, 10])

for i in range(50):
    quadrotor.step(u)

In [4]:
fig = mp.figure.Figure(figsize=[8, 8])
mp.backends.backend_agg.FigureCanvasAgg(fig)
ax = fig.add_subplot(111, autoscale_on=False, xlim=[-2, 12], ylim=[-2, 12])
ax.grid()

#create the quadrotor
center, = ax.plot([], [], 'k', marker="o")
lines = []
    
for i in range(8):
    line, = ax.plot([], [], 'k', lw=2)
    lines.append(line)

In [5]:
def drawQuadrotor(quadrotor, ax, center, lines, t):
    r = quadrotor.r
    h = r/2
    x, y, theta, u, v, omega = quadrotor.x
    
    for line in lines: #reset all lines
        line.set_data([],[])
    
    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    t = np.array([[x], [y]])
    
    A = np.array([-r, h/2])
    B = np.array([ r, h/2])
    C = np.array([ r,-h/2])
    D = np.array([-r,-h/2])
    
    E = np.array([-r+h/2, h/2])
    F = np.array([-r+h/2, h   ])
    G = np.array([ r-h/2, h/2])
    H = np.array([ r-h/2, h   ])
    
    I = np.array([F[0]-h*np.cos(np.pi*t), F[1]])
    J = np.array([F[0]+h*np.cos(np.pi*t), F[1]])
    K = np.array([H[0]-h*np.cos(np.pi*t), H[1]])
    L = np.array([H[0]+h*np.cos(np.pi*t), H[1]])
    
    coords = np.vstack([A, B, C, D, E, F, G, H, I, J, K, L])
    coords = coords.T
    
    coords = R @ coords + t
    
    A = coords[:, 0]
    B = coords[:, 1]
    C = coords[:, 2]
    D = coords[:, 3]
    E = coords[:, 4]
    F = coords[:, 5]
    G = coords[:, 6]
    H = coords[:, 7]
    I = coords[:, 8]
    J = coords[:, 9]
    K = coords[:, 10]
    L = coords[:, 11]
    
    center.set_data([x], [y])
    
    lines[0].set_data([A[0], B[0]], [A[1], B[1]])
    lines[1].set_data([B[0], C[0]], [B[1], C[1]])
    lines[2].set_data([C[0], D[0]], [C[1], D[1]])
    lines[3].set_data([A[0], D[0]], [A[1], D[1]])
    
    lines[4].set_data([E[0], F[0]], [E[1], F[1]])
    lines[5].set_data([G[0], H[0]], [G[1], H[1]])
    
    lines[6].set_data([I[0], J[0]], [I[1], J[1]])
    lines[7].set_data([K[0], L[0]], [K[1], L[1]])
    
    return lines

In [16]:
def animate_quadrotor(quadrotor, controller, horizon):
    dt = quadrotor.dt
    
    fig = mp.figure.Figure(figsize=[12, 12])
    mp.backends.backend_agg.FigureCanvasAgg(fig)
    ax = fig.add_subplot(111, autoscale_on=False, xlim=[-2, 42], ylim=[-2, 42])
    ax.grid()

    #create the quadrotor
    center, = ax.plot([], [], 'k', marker="o")
    lines = []

    for i in range(8):
        line, = ax.plot([], [], 'k', lw=2)
        lines.append(line)
    
    # simulate with controller
    def animate(i):
        nonlocal lines
        t = dt * i
        lines = drawQuadrotor(quadrotor, ax, center, lines, t)
        u = controller(quadrotor.x)
        quadrotor.step(u)
        return lines
        
    def init():
        return animate(0)
    
    ani = animation.FuncAnimation(fig, animate, np.arange(0, horizon),
        interval=1000*dt, blit=True, init_func=init)
    plt.close(fig)
    plt.close(ani._fig)
    IPython.display.display_html(IPython.core.display.HTML(ani.to_html5_video()))

In [20]:
def controller(x):
    return np.random.randint(0, 20, 2)

In [21]:
quadrotor = Quadrotor(m, I, r)

animate_quadrotor(quadrotor, controller, 1000)

Your browser does not support the video tag.